In [1389]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import datasets
from sklearn.metrics import accuracy_score

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [1347]:
df19 = pd.read_csv("player_stats_2018_2019.csv")

In [1348]:
df19["season"] = 19

In [1349]:
df18 = pd.read_csv("player_stats_2017_2018.csv")

In [1350]:
df18["season"] = 18

In [1351]:
df17 = pd.read_csv("player_stats_2016_2017.csv")

In [1352]:
df17["season"] = 17

In [1353]:
df16 = pd.read_csv("player_stats_2015_2016.csv")

In [1354]:
df16["season"] = 16

In [1355]:
df15 = pd.read_csv("player_stats_2014_2015.csv")

In [1356]:
df15["season"] = 15

In [1357]:
df14 = pd.read_csv("player_stats_2013_2014.csv")

In [1358]:
df14["season"] = 14

In [1359]:
df13 = pd.read_csv("player_stats_2012_2013.csv")

In [1360]:
df13["season"] = 13

In [1361]:
df12 = pd.read_csv("player_stats_2011_2012.csv")

In [1362]:
df12["season"] = 12

In [1363]:
df11 = pd.read_csv("player_stats_2010_2011.csv")

In [1364]:
df11["season"] = 11

In [1365]:
df10 = pd.read_csv("player_stats_2009_2010.csv")

In [1366]:
df10["season"] = 10

In [1367]:
df9 = pd.read_csv("player_stats_2008_2009.csv")

In [1368]:
df9["season"] = 9

In [1369]:
df8 = pd.read_csv("player_stats_2007_2008.csv")

In [1370]:
df8["season"] = 8

In [1371]:
df7 = pd.read_csv("player_stats_2006_2007.csv")

In [1372]:
df7["season"] = 7

In [1373]:
df6 = pd.read_csv("player_stats_2005_2006.csv")

In [1374]:
df6["season"] = 6

In [1375]:
master = pd.DataFrame()

In [1376]:
master = master.append([df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19], ignore_index=True, sort=False)

In [1377]:
master.shape

(5822, 53)

In [1378]:
master = master.sort_values(by=['season','Rk'], ascending=[0,1])

In [1379]:
#CALCULATE ADDITIONAL DATA POINTS FOR FEATURES
for index, row in master.iterrows():
    current_player = row["Player"]
    current_season = row["season"]
    prev_season = current_season - 1
    if(current_player in master[master["season"]==prev_season]["Player"].values):
        
        #FEATURE 1: OPPORTUNITY - DATA POINTS
        
        # DELTA USG
        prev_usg = master[master["season"]==prev_season][master["Player"]==current_player]["USG%"]
        current_usg = row["USG%"]                                                 
        delta_usg = float(current_usg) - float(prev_usg)
        master.loc[index,"Delta USG%"] = delta_usg
        # DELTA AST
        prev_ast = master[master["season"]==prev_season][master["Player"]==current_player]["AST%"]
        current_ast = row["AST%"] 
        delta_ast = float(current_ast) - float(prev_ast)
        master.loc[index,"Delta AST%"] = delta_ast
        
        #FEATURE 2: IMPORTANT DELTAS - DATA POINTS
        
        #DELTA POINTS
        prev_pts = master[master["season"]==prev_season][master["Player"]==current_player]["PTS"]
        current_pts = row["PTS"] 
        delta_pts = float(current_pts) - float(prev_pts)
        master.loc[index,"Delta PTS"] = delta_pts
        #DELTA TOTAL REBOUNDS
        prev_trb = master[master["season"]==prev_season][master["Player"]==current_player]["TRB"]
        current_trb = row["TRB"] 
        delta_trb = float(current_trb) - float(prev_trb)
        master.loc[index,"Delta TRB"] = delta_trb
        #DELTA ASSISTS
        prev_asts = master[master["season"]==prev_season][master["Player"]==current_player]["AST"]
        current_asts = row["AST"] 
        delta_asts = float(current_asts) - float(prev_asts)
        master.loc[index,"Delta AST"] = delta_asts
        #DELTA STL
        prev_stl = master[master["season"]==prev_season][master["Player"]==current_player]["STL"]
        current_stl = row["STL"] 
        delta_stl = float(current_stl) - float(prev_stl)
        master.loc[index,"Delta STL"] = delta_stl
        #DELTA STL
        prev_blk = master[master["season"]==prev_season][master["Player"]==current_player]["BLK"]
        current_blk = row["BLK"] 
        delta_blk = float(current_blk) - float(prev_blk)
        master.loc[index,"Delta BLK"] = delta_blk
        

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:

In [1380]:
# Feature 1 : Opportunity
master["Opportunity"] = master["Delta USG%"] + (master["Delta AST%"]*-1)

In [1381]:
# Feature 2: Important Deltas
master["Impt Deltas"] = master["Delta PTS"] + master["Delta TRB"] + master["Delta AST"]

In [1382]:
# Feature 3: Defensive Ability
master["Defensive Ability"] = ( master["STL"] + master["BLK"] ) / master["PF"]

In [1383]:
# Feature 4: SOMETHING
master["Total Mins"] = master["MP"] * master["G"]

In [1384]:
# DATA CLEANING

# Remove any players with current season less than 1000 mins and previous season less than 250 mins
dropped = 0
print(master.shape)
master = master[master["Total Mins"] > 250]
print(master.shape)
for index, row in master.iterrows():
    current_player = row["Player"]
    current_season = row["season"]
    prev_season = current_season - 1
    if(current_player in master[master["season"]==prev_season]["Player"].values):
        if (row["Total Mins"] < 1000):
            master.drop(index, inplace=True)
            dropped += 1

print("dropped this many < 1000: " + str(dropped) + "rows")


(5822, 64)
(4772, 64)
dropped this many < 1000: 733rows


In [1385]:
master.shape

(4039, 64)

In [1392]:
# Define feature set 1
feature_set_1 = pd.DataFrame
feature_set_1 = master[['Player','Age','Total Mins','PTS','TRB','AST','STL','BLK','TS%','WS','Opportunity','Defensive Ability','MIP']]

In [1393]:
# Define feature set 2
feature_set_2 = pd.DataFrame
feature_set_2 = master[['Player','Age','Total Mins','Defensive Ability','FG%','3P%','FT%','WS','MIP']]

In [1394]:
# Define feature set 3
feature_set_3 = pd.DataFrame
feature_set_3 = master[['Player','Age','Total Mins','Delta PTS','Delta TRB','Delta AST','Delta STL', 'Delta BLK','MIP']]


In [1395]:
%store master

Stored 'master' (DataFrame)


In [1396]:
%store feature_set_1
%store feature_set_2
%store feature_set_3

Stored 'feature_set_1' (DataFrame)
Stored 'feature_set_2' (DataFrame)
Stored 'feature_set_3' (DataFrame)
